In [31]:
import geopandas as gpd
import requests
import json
from shapely.geometry import Polygon
from geojson import Feature, FeatureCollection
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd

In [23]:
def up_load_shp(path):
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [24]:
shapefile_path='../ags-iplan-data-fetcher-python/polygon_input/polygon.shp'
polygon_input=up_load_shp(shapefile_path)

In [25]:
minx = polygon_input.bounds.minx[0]
miny = polygon_input.bounds.miny[0]
maxx = polygon_input.bounds.maxx[0]
maxy = polygon_input.bounds.maxy[0]

In [26]:
url = r'https://ags.iplan.gov.il/arcgisiplan/rest/services/PlanningPublic/Xplan/MapServer/1/query?f=json&where=pl_area_dunam%20<%3D15&returnGeometry=true&geometry=%7B%22xmin%22%3A{}%2C%22ymin%22%3A{}%2C%22xmax%22%3A{}%2C%22ymax%22%3A{}%2C%22spatialReference%22%3A%7B%22wkid%22%3A2039%7D%7D&geometryType=esriGeometryEnvelope&inSR=2039&outFields=pl_number%2Cpl_name%2Cpl_url%2Cquantity_delta_120%2Cstation_desc%2Cplan_county_name&orderByFields=pl_number&outSR=2039'.format(minx, miny, maxx, maxy)
response = requests.get(url)
response_json = response.json()

In [27]:
response_data = json.dumps(response_json)
dict = json.loads(response_data)

xplan_plans_data = dict['features']
# xplan_plans_data
xplan_plans_data_filtered = []

for plan in xplan_plans_data:
    interior_rings = plan['geometry']['rings'][1:]
    exterior_ring=plan['geometry']['rings'][0]
    polygon = Polygon(exterior_ring, holes=interior_rings)
    if polygon_input.contains(polygon.centroid).bool():
        xplan_plans_data_filtered.append(plan)

C:\Users\dpere\AppData\Local\Temp\ipykernel_24200\1718570928.py:12: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if polygon_input.contains(polygon.centroid).bool():


In [28]:
chrome_options = webdriver.ChromeOptions()

prefs = {"profile.default_content_setting_values.notifications" : 2}

chrome_options.add_experimental_option("prefs",prefs)

PATH = 'chromedriver-win64/chromedriver.exe'

service = Service(executable_path=PATH)

mydivs_list = []

new_plans_data = []

browser = Browser('chrome')

for plan in xplan_plans_data_filtered:
    print(counter)
    lis_elements = []
    browser.visit(plan['attributes']['pl_url'])
    time.sleep(7)
    buttons = browser.find_by_tag('button')
    for button in buttons:
        if button.text == 'נתונים נוספים':
            button.click()
            
    soup = BeautifulSoup(browser.html, 'html.parser')
    mydivs = soup.find_all("li", {"class": "sv4-icon-arrow uk-open uk-hide-arrow ng-star-inserted"})
    mydivs_list.append(mydivs)

    for divs in mydivs:
        div = divs.find_all('div', attrs={'class': 'uk-accordion-content uk-margin-remove'})

    results = div[0].find_all('ul')

    for result in results:
        li = result.find_all('li')
        lis_elements.append(li)
        
    obj = {}
    for element in lis_elements:
        quantitative_data_main_header = element[0].find_all('div', {'class': 'uk-width-expand ng-star-inserted'})[0].text.strip()
        quantitative_data_main_value = element[0].find_all('div', {'class': 'uk-width-1-2 uk-text-left'})[0].find_all('b')[0].text
        obj[quantitative_data_main_header] = quantitative_data_main_value

        plan['attributes'].update(obj)    
browser.quit()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
[{'attributes': {'pl_number': '101-0056499', 'pl_name': 'תוספת בניין חדש ושימור מבנה בשכונת בית יעקב', 'pl_url': 'https://mavat.iplan.gov.il/SV4/1/1000215815/310', 'quantity_delta_120': 34.0, 'station_desc': 'אישור', 'plan_county_name': 'ירושלים'}, 'geometry': {'rings': [[[220154.55009999964, 632565.5000999998], [220147.2100999998, 632538.5601000004], [220106.34649999999, 632549.8703000005], [220108.28129999992, 632556.4173000008], [220105.82849999983, 632557.0941000003], [220102.9093000004, 632557.8118999992], [220108.5625, 632578.5381000005], [220114.3350999998, 632576.9014999997], [220154.55009999964, 632565.5000999998]]]}}, {'attributes': {'pl_number': '101-0063933', 'pl_name': 'מבנה למגורים משרדים מסחר שטחי ציבור וחניון תת"ק במרכז העיר ירושלים', 'pl_url': 'https://mavat.iplan.gov.il/SV4/1/1000222801/310', 'quantity_delta_120': 12.0, 'station_desc': 'אישור', 'plan_county_name': 'ירושלים'}, 'geometry

In [33]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

'240325'

In [34]:
features = []

key_mapping = {
    'חדרי מלון / תיירות (חדר)': 'hotel_rooms_room',
    'חדרי מלון / תיירות (מ"ר)': 'hotel_rooms_square_meters',
    'מבני ציבור (מ"ר)': 'public_buildings_square_meters',
    'מגורים (יח"ד)': 'residence_housing_units',
    'מגורים (מ"ר)': 'residence_square_meters',
    'מסחר (מ"ר)': 'trade_square_meters',
    'תעסוקה (מ"ר)': 'employment_square_meters',
    'דירות קטנות (יח"ד)': 'small_apartments_units',
    'דירות להשכרה (יח"ד)': 'apartments_for_rent_units'
}

for plan in xplan_plans_data:
        # Create a new dictionary to store the updated attributes
    new_attributes = {}
    # Loop over the attributes dictionary in each item
    for key, value in plan['attributes'].items():
        # Check if the key needs to be replaced
        if key in key_mapping:
            # Replace the key
            new_key = key_mapping[key]
            # Add the new key with the same value
            new_attributes[new_key] = value
        else:
            # If key doesn't need to be replaced, keep it as it is
            new_attributes[key] = value
    # Update the attributes with the new dictionary
    plan['attributes'] = new_attributes

    interior_rings = plan['geometry']['rings'][1:]
    exterior_ring=plan['geometry']['rings'][0]
    polygon = Polygon(exterior_ring, holes=interior_rings)
    if polygon_input.contains(polygon.centroid).bool():
        feature = Feature(geometry=polygon, properties=plan['attributes'])
        features.append(feature)
        
crs = {
    "type": "name",
    "properties": {
        "name": "EPSG:2039"
    }
}
        
feature_collection = FeatureCollection(features)

gdf = gpd.GeoDataFrame.from_features(feature_collection, crs='EPSG:2039')
gdf.to_file(r'my_polygon/{}_polygon.shp'.format(file_date), encoding='utf-8')
gdf


C:\Users\dpere\AppData\Local\Temp\ipykernel_24200\2881014987.py:35: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if polygon_input.contains(polygon.centroid).bool():
C:\Users\dpere\AppData\Local\Temp\ipykernel_24200\2881014987.py:49: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(r'my_polygon/{}_polygon.shp'.format(file_date), encoding='utf-8')


,geometry,pl_number,pl_name,pl_url,quantity_delta_120,station_desc,plan_county_name,hotel_rooms_room,hotel_rooms_square_meters,public_buildings_square_meters,residence_housing_units,residence_square_meters,trade_square_meters,employment_square_meters,small_apartments_units,apartments_for_rent_units
0,"POLYGON ((220566.138 632378.646, 220560.005 63...",101-0135004,"מתחם בנין-מסחר, תעסוקה, מגורים ומלונאות, רח' י...",https://mavat.iplan.gov.il/SV4/1/1000262173/310,256.0,אישור,ירושלים,228,"13,458","1,643.5",381,"33,474","11,529.5","7,894.5",NaN,NaN
1,"POLYGON ((220445.730 632203.382, 220448.906 63...",101-0141283,מבנה מגורים ברחוב מסילת ישרים,https://mavat.iplan.gov.il/SV4/1/1000266189/310,5.0,אישור,ירושלים,NaN,NaN,NaN,12,785,50.5,50.5,11,NaN
2,"POLYGON ((220101.030 632454.650, 220092.400 63...",101-0149633,"תוספת 6 יח""ד ושתי קומות חדשות ברחוב אגריפס 76",https://mavat.iplan.gov.il/SV4/1/1000272985/310,6.0,אישור,ירושלים,NaN,NaN,NaN,11,490.88,337.42,NaN,11,NaN
3,"POLYGON ((220395.445 632177.474, 220392.730 63...",101-0175604,"תוספת יחידות דיור ברח' אבולפעיה מס' 36, ירושלים",https://mavat.iplan.gov.il/SV4/1/1000288139/310,2.0,אישור,ירושלים,NaN,NaN,NaN,8,280,NaN,NaN,8,NaN
4,"POLYGON ((220442.006 632598.098, 220431.245 63...",101-0206748,"תוספת 8 יח""ד חדשות ברח' אורנשטיין 9, ירושלים",https://mavat.iplan.gov.il/SV4/1/1000304605/310,8.0,אישור,ירושלים,NaN,NaN,NaN,19,"1,148.01",NaN,NaN,NaN,NaN
5,"POLYGON ((220341.890 632497.500, 220393.220 63...",101-0217885,"תוספת קומות, יח""ד ושטחי מסחר מלונאות/משרדים, ר...",https://mavat.iplan.gov.il/SV4/1/1000310637/310,10.0,אישור,ירושלים,80,"1,952",NaN,158,"11,550.9","1,997.1",NaN,NaN,NaN
6,"POLYGON ((220659.255 632282.737, 220657.603 63...",101-0223347,תוספת קומות למגורים מעל בניין קיים ברח' יפו י-ם,https://mavat.iplan.gov.il/SV4/1/1000313327/310,42.0,אישור,ירושלים,NaN,NaN,NaN,42,"2,137.62",644.39,NaN,42,NaN
7,"POLYGON ((220387.408 632422.600, 220395.709 63...",101-0301929,"מתחם כי""ח",https://mavat.iplan.gov.il/SV4/1/1000351135/310,184.0,אישור,ירושלים,70,"3,100",700,184,"16,600","14,157","3,100",44,NaN
8,"POLYGON ((220690.420 632340.280, 220683.238 63...",101-0364935,"תוספת יח""ד וקומות ברח' יפו 60, ירושלים",https://mavat.iplan.gov.il/SV4/1/1000382663/310,14.0,אישור,ירושלים,NaN,NaN,NaN,15,746.1,339.4,NaN,NaN,NaN
9,"POLYGON ((220496.603 632366.137, 220496.753 63...",101-0370825,7 קומות מגורים מעל קומת מסחר קיימת ברח' יפו 9...,https://mavat.iplan.gov.il/SV4/1/1000385805/310,56.0,אישור,ירושלים,NaN,NaN,NaN,58,"1,898.79","1,547.5",NaN,58,NaN
